<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/WE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz

--2020-08-08 00:33:07--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39356138 (38M) [application/octet-stream]
Saving to: ‘Musical_Instruments_5.json.gz.1’

Musical_Instruments 100%[===================>]  37.53M  56.8MB/s    in 0.7s    

2020-08-08 00:33:07 (56.8 MB/s) - ‘Musical_Instruments_5.json.gz.1’ saved [39356138/39356138]



In [27]:
import gzip
import json

In [28]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        d = json.loads(l)

        yield d


In [45]:
data = parse("Musical_Instruments_5.json.gz")



labels = []
reviews = []


In [48]:
for i in range(10000):
  L = next(data)
  if "reviewText" in L :
    if "overall" in L :
      if L["overall"] > 2.0:
        labels.append(1)
        
      else:
        labels.append(0)
      reviews.append(L["reviewText"])
      

In [50]:
import numpy as np
np.mean(labels)

0.9578421578421579

In [52]:
from keras.preprocessing.text import Tokenizer
import numpy as np

max_words = 10000 # We will only consider the 10K most used words in this dataset

tokenizer = Tokenizer(num_words=max_words) # Setup
tokenizer.fit_on_texts(reviews) # Generate tokens by counting frequency
sequences = tokenizer.texts_to_sequences(reviews) # Turn text into sequence of numbers

In [53]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100 # Make all sequences 100 words long
data = pad_sequences(sequences, maxlen=maxlen)
print(data.shape) # We have 25K, 100 word sequences now

(10010, 100)


In [55]:
labels = np.asarray(labels)

# Shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

training_samples = 20000  # We will be training on 10K samples
validation_samples = 5000  # We will be validating on 10000 samples

# Split data
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [56]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

embedding_dim = 50

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           500000    
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                160032    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 660,065
Trainable params: 660,065
Non-trainable params: 0
_________________________________________________________________


In [67]:
 import tensorflow as tf
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.RootMeanSquaredError(),"acc"])

In [68]:
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Epoch 1/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0016 - root_mean_squared_error: 0.0183 - acc: 0.9996
Epoch 2/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0014 - root_mean_squared_error: 0.0173 - acc: 0.9996
Epoch 3/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0014 - root_mean_squared_error: 0.0183 - acc: 0.9996
Epoch 4/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0013 - root_mean_squared_error: 0.0173 - acc: 0.9996
Epoch 5/50
313/313 [==============================] - 2s 8ms/step - loss: 0.0015 - root_mean_squared_error: 0.0186 - acc: 0.9995
Epoch 6/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0012 - root_mean_squared_error: 0.0190 - acc: 0.9994
Epoch 7/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0017 - root_mean_squared_error: 0.0184 - acc: 0.9995
Epoch 8/50
313/313 [==============================] - 3s 8ms/step - loss: 0.0015 - root_mean_squa

In [70]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-08 00:49:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-08 00:49:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-08 00:49:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [73]:
import zipfile
with zipfile.ZipFile("/content/glove.6B.zip", 'r') as zip_ref:
    zip_ref.extractall("./content")

In [76]:
from tqdm import tqdm

In [77]:
import os


print('Loading word vectors')
embeddings_index = {} # We create a dictionary of word -> embedding
f = open( 'glove.6B.100d.txt') # Open file

# In the dataset, each line represents a new word embedding
# The line starts with the word and the embedding values follow
for line in tqdm(f):
    values = line.split()
    word = values[0] # The first value is the word, the rest are the values of the embedding
    embedding = np.asarray(values[1:], dtype='float32') # Load embedding
    embeddings_index[word] = embedding # Add embedding to our embedding dictionary
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2704it [00:00, 27038.09it/s]

Loading word vectors


400000it [00:13, 30628.70it/s]

Found 400000 word vectors.


In [78]:
# Create a matrix of all embeddings
all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean() # Calculate mean
emb_std = all_embs.std() # Calculate standard deviation
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.004451992, 0.4081574)

In [79]:
embedding_dim = 100 # We now use larger embeddings

word_index = tokenizer.word_index
nb_words = min(max_words, len(word_index)) # How many words are there actually

# Create a random matrix with the same mean and std as the embeddings
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= max_words: 
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings_index.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [80]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights = [embedding_matrix], trainable = False))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


In [84]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])



In [86]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=100,
    verbose=1,
    validation_data=(x_val, y_val),
    
)


Epoch 1/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0063 - acc: 0.9983
Epoch 2/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0073 - acc: 0.9977
Epoch 3/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0088 - acc: 0.9971
Epoch 4/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0053 - acc: 0.9985
Epoch 5/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0145 - acc: 0.9961
Epoch 6/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0052 - acc: 0.9981
Epoch 7/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0056 - acc: 0.9987
Epoch 8/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0052 - acc: 0.9981
Epoch 9/100
313/313 [==============================] - 2s 5ms/step - loss: 0.0049 - acc: 0.9985
Epoch 10/100
313/313 [==============================] - 2s 5ms/step - loss: 0.0045 - acc: 0.9987
Epoch 11/100
313/313 [=================

In [88]:
# Demo on a negative text
my_text = 'The bleak economic outlook will force many small businesses into bankruptcy.'

seq = tokenizer.texts_to_sequences([my_text])
print('raw seq:',seq)
seq = pad_sequences(seq, maxlen=maxlen)
print('padded seq:',seq)
prediction = model.predict(seq)
print('positivity:',prediction)

raw seq: [[1, 48, 1078, 171, 205, 7975, 137]]
padded seq: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    1   48 1078  171  205
  7975  137]]
positivity: [[0.99954164]]
